My First Kaggle Kernel with lot of enthusiasm! Eager to listen from all Kagglers!  All suggestions, improvements, ideas, questions and queries are well come.
The signal is sum of sinusoidal wave, other interfering waves, random disturbances and measurement noise. The sinusoidal wave is the indented signal and rest is deviations from various sources. One of the sources is fault on line. Using superimposition of wave theory we can subtract the sinusoidal wave from signal and get all deviation. From the difference in faulty and non-faulty residuals we can get patterns exhibited by fault.
To reach this goal, we divide it following sub tasks,
1.	Smooth the signal so we can fit sine wave
2.	Fit the sine wave
3.	Calculated residual
4.	Compare some good and faulty signals in term of residuals.

The optimize library from scipy is used for sine wave fitting. The find_peak function from scipy.signal is used to identify peaks and Kmeans from sklearn.cluster is used for clustering those peaks. The smooth function denoises data and sine_func implements the sine function.


In [ ]:
import os
import pyarrow.parquet as pq
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy import optimize
from scipy.signal import find_peaks
from sklearn.cluster import KMeans

#The number of sample in every record
dataPoints=800000
def smooth(y, box_pts):
    box = np.ones(box_pts)/box_pts
    y_smooth = np.convolve(y, box, mode='same')
    return y_smooth

def sine_func(x, a, b, c,d):
    return a * np.sin((b*x)+c)+d

Let us read a signal and plot it to see how it looks.

In [ ]:
t = pd.read_parquet('../input/train.parquet', engine='pyarrow', columns=['575'])  #8021 8031 8081  
plt.figure(figsize=(24, 10))
plt.plot(t[:],label='Raw series')
plt.legend(loc='best')
plt.show()

The fitting of sine wave is a challenging task, especially to noisy data. To smooth data the numpy's convolve method is used. The second parameter decided the convolution window.  The smoothed wave is clearly a sine wave. The sine wave is fitted to smoothed data.

In [ ]:
y_data = t[0:dataPoints].values.reshape(dataPoints)
x_data = np.linspace(0,1, num=dataPoints).T.reshape(dataPoints)
y_data_smooth = smooth(y_data, 10000)
plt.figure(figsize=(24, 10))
plt.plot( y_data,'r+')
plt.plot(y_data_smooth,"b-",label='Smoothed Data')
plt.legend(loc='best')
plt.show()

To fit sine wave, one need to provide initial guess values for parameters like amplitude, phase and shift. Other parameters are known from context we need to find out phase. I will discuss the sine equation and other parameters while fitting sine wave. Now we focus on phase. Phase is the position of a point in time on a waveform cycle.(Further reference https://en.wikipedia.org/wiki/Phase_(waves) ) We estimate the phase  by locating the positive peak. To identify peaks we use find_peaks method from scipy.signal library. We specify width to avoid sudden fluctuations and smaller peaks.

In [ ]:
peaks, _ = find_peaks(y_data_smooth,threshold =(0.00001,0.0001),width = 100)
plt.plot(y_data_smooth)
plt.plot(peaks, y_data_smooth[peaks], "x")
plt.plot(np.zeros_like(y_data_smooth), "--", color="gray",label='Peaks')
plt.legend(loc='best')
plt.show()

In [ ]:
The more than two peaks are found. We cluster this peak into two clusters and take average 
for finding their position on x-axis. Kmeans cluster with 2 numbers of clusters returns the cluster 
index for every peak. We check which is peak is positive by comparing them and take average of 
peak positions for it. From this location of positive peak we calculated the phase with below reasoning.

The entire signel is capturing a full wave i.e 360 degress or 2 pi in 800000 time slices. 
So the every displacement counts for 2*pi/800000. 
The positive peack will appear at pi/2 for zero phase. and hence
phase = (((800000- posivePeackLoc)+200000)% 800000)/800000*np.pi/2         ---phase formula

The sine wave has equation,
y(t) = A\sin(2 \pi f t + \varphi) = A\sin(\omega t + \varphi)
where:

A = the amplitude, the peak deviation of the function from zero.
f = the ordinary frequency, the number of oscillations (cycles) that occur each second of time.
ω = 2πf, the angular frequency, the rate of change of the function argument in 
        units of radians per second 
{\displaystyle \varphi } \varphi  = the phase, specifies (in radians) where in its cycle 
        the oscillation is at t = 0.    When {\displaystyle \varphi } \varphi  is non-zero, 
    the entire waveform appears to be shifted in time by the amount {\displaystyle \varphi } 
    \varphi /ω seconds. A negative value represents a delay, and a positive value represents an advance.

We also add a displacement parameter so the fitted curve can move horizontally.

The intial guasses are below,
A = 20 visual inspection  and problem description.
f = 1/800000. There are 800000 samples per full cycle.
ω = 2πf = 4*np.pi/800000
phase = (((800000- posivePeackLoc)+200000)% 800000)/800000*np.pi/2         ---phase formula 

The fitted wave is plotted in blue over red raw signal. Good fit!

In [ ]:
km = KMeans(n_clusters=2)
km.fit(peaks.reshape(-1,1))
clu = km.predict(peaks.reshape(-1,1))

if ( np.mean(y_data_smooth[peaks[clu==1]]) > np.mean(y_data_smooth[peaks[clu==0]])) :
        posivePeackLoc = np.mean(peaks[clu==1])
else:
        posivePeackLoc = np.mean(peaks[clu==0])

y_data = t[0:dataPoints].values.reshape(dataPoints)
x_data = np.linspace(0,1, num=dataPoints).T.reshape(dataPoints)
y_data[y_data>45]=45
y_data[y_data<-45]=-45
phase = (((800000- posivePeackLoc)+200000)% 800000)/800000*np.pi/2
params, params_covariance = optimize.curve_fit(sine_func, x_data, y_data,
                                               p0=[20, 4*np.pi/800000, phase,5])

plt.figure(figsize=(24, 10))
plt.plot( y_data,'r+')
plt.plot(x_data*dataPoints, sine_func(x_data, params[0], params[1],params[2],params[3]),
         label='Fitted Sine Wave')
plt.legend(loc='best')
plt.show()

Now getting residul is very easy. Just substract the fitted sine wave from raw data. Below is residual for non fault case. (signal id 575)

In [ ]:
residual=y_data-sine_func(x_data, params[0], params[1],params[2],params[3])
plt.figure(figsize=(24, 10))
plt.plot(x_data*dataPoints, y_data-sine_func(x_data, params[0], params[1],params[2],params[3]),
         label='Residual')
plt.legend(loc='best')
plt.show()

[Imgur](https://i.imgur.com/X1PnOr3.png)

These lines are disturbances. The circular patterns are due to fittment error at peak. Please compare this image with the Residual graph for faulty case. (signal id 7919) shown below,

[Imgur](https://i.imgur.com/ERYuetn.png)

The two images indicate 1) the frequencies of vertical lines is more in faulty lines.
                                            2) The width of these lines is more in faulty lines.
                                            3) The height of these lines is more in faulty lines.
                                            4) The fitment error is dominated in non-faulty lines.
   These study suggests that we can use some image processing, CNN, LSTM to use visual clues. Also we can extract has disturbance frequencies, time length, density for feature selection.